# Implementaion of [1]

- The paper's MSE is diffferent from our result. It is due to dephasing, which is represented by $\alpha$ and $\beta$.
- The ideal result is obtained in the case of $\alpha=0$ and $\beta=1$.

- **MSE is the average variance, not standard deviation.**

- The standard deviation informs how faithful is an estimation.

In [113]:
import numpy as np
from numpy import pi, cos, sqrt, log
from matplotlib import pyplot as plt
import statistics as stats
from tqdm import tqdm

In [117]:
f0 = 100e6
w0 = 2 * pi * f0
tau = pi/w0/1.5
alpha = 0.25
beta = 0.67

In [121]:
# Trigonometric polynomial of the coin probability

def tripoly1sample(r, m):
    lst = [0] * (2*m + 1)
    lst[0] = r * beta * 1/2
    lst[2*m] = r * beta * 1/2
    lst[m] = 1 + alpha
    
    return np.poly1d(lst)

## Bayesian Estimation (n=1, m=1, 2, ..., N)

In [120]:
N = 100

def toss(m):
    p = .5*(1 + alpha + beta * cos(w0 * m * tau))
    return np.random.choice([1, -1], p=[p, 1-p])

ml = list(range(1, N+1))
ml = [round((1.05)**i) for i in range(1, N+1)]
# ml = list(range(1, (N+1)//3))*3
rl = [toss(m) for m in ml]

tripolyN = 1
for m, r in zip (ml, rl):
    tripolyN *= tripoly1sample(r, m)
    
coeff = tripolyN.c
M = (len(coeff)-1)//2
coeff = np.divide(coeff, coeff[M])

coeff_avg = []
for n in range(-M, M+1):
    if n == 0:
        coeff_avg.append(pi/2)
    else:
        coeff_avg.append(((-1)**n-1)/n**2/pi)

coeff_sqavg = []
for n in range(-M, M+1):
    if n == 0:
        coeff_sqavg.append(pi**2/3)
    else:
        coeff_sqavg.append((-1)**n*2/n**2)

avg = np.dot(coeff, coeff_avg)/2/pi/tau
sqavg = np.dot(coeff, coeff_sqavg)/(2*pi*tau)**2
sd = sqrt(sqavg-avg**2)

print(f"Avg: {avg/1e6:.6f}")
print(f"Std: {sd/1e6:.6f}")

Avg: 100.072185
Std: 0.336132


## Adaptive Method

# Try on [2]

# Reference

[1] Sergeevich, A., Chandran, A., Combes, J., Bartlett, S. D., & Wiseman, H. M. (2011). Characterization of a qubit Hamiltonian using adaptive measurements in a fixed basis. Physical Review A - Atomic, Molecular, and Optical Physics, 84(5), 052315. https://doi.org/10.1103/PhysRevA.84.052315

[2] Ferrie, C., Granade, C. E., & Cory, D. G. (2013). How to best sample a periodic probability distribution, or on the accuracy of Hamiltonian finding strategies. Quantum Information Processing 2012 12:1, 12(1), 611–623. https://doi.org/10.1007/S11128-012-0407-6